# Module 05: External API Integration

**Difficulty**: ⭐⭐⭐
**Estimated Time**: 60 minutes
**Prerequisites**: [Module 01](01_metadata_management.ipynb), [Module 03](03_album_art_management.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Query MusicBrainz database to find correct metadata for your music
2. Automatically download album art from online sources
3. Enrich existing metadata with information from music databases
4. Correct misspelled or inconsistent artist/album names
5. Batch process files to enhance metadata automatically
6. Understand API rate limiting and best practices

## Why External APIs Matter

Your music files may have:

- **Missing metadata**: No artist, album, or year information
- **Incorrect metadata**: Typos or wrong information
- **Inconsistent metadata**: "Beatles" vs "The Beatles" vs "beatles"
- **Missing album art**: No embedded artwork

**External music databases can help fix these issues automatically!**

## What This Module Covers

1. **MusicBrainz Integration**
   - Search for recordings, artists, and albums
   - Get authoritative metadata (official spellings, release dates)
   - Retrieve album art URLs from Cover Art Archive

2. **Album Art Download**
   - Fetch high-quality album artwork
   - Save or embed artwork automatically
   - Handle different image sizes and formats

3. **Metadata Enrichment**
   - Add missing genre information
   - Fill in release years
   - Standardize artist/album names

4. **API Best Practices**
   - Respect rate limits (MusicBrainz: 1 request/second)
   - Cache results to avoid redundant queries
   - Handle API errors gracefully

---

## Setup

Import required libraries and configure settings.

**Note**: This module requires `musicbrainzngs` for MusicBrainz API and `requests` for downloading. Install if needed:
```bash
pip install musicbrainzngs requests
```

In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Tuple
import time
import json

# Third-party imports
import pandas as pd
from IPython.display import display, Image as IPythonImage
from mutagen import File
from mutagen.id3 import ID3

# API libraries
try:
    import musicbrainzngs as mb
    MB_AVAILABLE = True
except ImportError:
    MB_AVAILABLE = False
    print("⚠️  musicbrainzngs not installed. MusicBrainz features will be disabled.")
    print("   Install with: pip install musicbrainzngs")

try:
    import requests
    REQUESTS_AVAILABLE = True
except ImportError:
    REQUESTS_AVAILABLE = False
    print("⚠️  requests not installed. Cannot download album art.")
    print("   Install with: pip install requests")

In [ ]:
# Configuration
MUSIC_LIBRARY_PATH = Path('../../music').resolve()

# Supported audio file extensions
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Configure MusicBrainz
if MB_AVAILABLE:
    mb.set_useragent(
        "MusicLibraryManager",
        "1.0",
        "https://github.com/yourusername/music-library-manager"  # Update with your info
    )
    # Rate limit: MusicBrainz allows 1 request per second
    MB_RATE_LIMIT = 1.0  # seconds between requests

# Cache for API results to avoid redundant queries
API_CACHE = {}

print(f"Music Library Path: {MUSIC_LIBRARY_PATH}")
print(f"MusicBrainz available: {MB_AVAILABLE}")
print(f"Requests available: {REQUESTS_AVAILABLE}")

---

## Part 1: MusicBrainz Metadata Lookup

Use MusicBrainz to find correct metadata for your music.

### What is MusicBrainz?

MusicBrainz is a **community-maintained open music encyclopedia** that:
- Contains information about millions of recordings
- Provides authoritative, standardized metadata
- Links to album artwork through Cover Art Archive
- Is completely free to use (with rate limits)

### How It Works

1. You provide: Artist name and/or track title
2. MusicBrainz searches its database
3. Returns: Official metadata, release dates, album art URLs, etc.

In [ ]:
def search_musicbrainz_recording(artist: str, title: str, limit: int = 5) -> List[Dict]:
    """
    Search MusicBrainz for a recording by artist and title.
    
    Args:
        artist: Artist name
        title: Track title
        limit: Maximum number of results to return
    
    Returns:
        List of matching recordings with metadata
    """
    if not MB_AVAILABLE:
        print("MusicBrainz not available. Install musicbrainzngs.")
        return []
    
    # Check cache first
    cache_key = f"recording:{artist}:{title}"
    if cache_key in API_CACHE:
        return API_CACHE[cache_key]
    
    try:
        # Search for the recording
        result = mb.search_recordings(
            artist=artist,
            recording=title,
            limit=limit
        )
        
        # Rate limiting - wait before next request
        time.sleep(MB_RATE_LIMIT)
        
        recordings = []
        for rec in result.get('recording-list', []):
            # Extract relevant metadata
            metadata = {
                'id': rec.get('id'),
                'title': rec.get('title'),
                'artist': rec.get('artist-credit-phrase', ''),
                'score': rec.get('ext:score', '0'),  # Match confidence (0-100)
                'length_ms': rec.get('length'),
            }
            
            # Get release information if available
            if 'release-list' in rec and rec['release-list']:
                release = rec['release-list'][0]
                metadata['album'] = release.get('title', '')
                metadata['release_id'] = release.get('id', '')
                metadata['date'] = release.get('date', '')
            
            recordings.append(metadata)
        
        # Cache the results
        API_CACHE[cache_key] = recordings
        
        return recordings
    
    except Exception as e:
        print(f"Error searching MusicBrainz: {e}")
        return []

In [ ]:
def get_release_info(release_id: str) -> Optional[Dict]:
    """
    Get detailed information about a release (album).
    
    Includes: Full metadata, track list, cover art URLs
    """
    if not MB_AVAILABLE:
        return None
    
    # Check cache
    cache_key = f"release:{release_id}"
    if cache_key in API_CACHE:
        return API_CACHE[cache_key]
    
    try:
        result = mb.get_release_by_id(
            release_id,
            includes=['artists', 'recordings', 'release-groups']
        )
        
        time.sleep(MB_RATE_LIMIT)
        
        release = result.get('release', {})
        
        info = {
            'id': release.get('id'),
            'title': release.get('title'),
            'artist': release.get('artist-credit-phrase', ''),
            'date': release.get('date', ''),
            'country': release.get('country', ''),
            'barcode': release.get('barcode', ''),
        }
        
        # Get release group for additional info
        if 'release-group' in release:
            rg = release['release-group']
            info['type'] = rg.get('type', '')  # Album, Single, EP, etc.
            info['release_group_id'] = rg.get('id', '')
        
        # Cache and return
        API_CACHE[cache_key] = info
        return info
    
    except Exception as e:
        print(f"Error getting release info: {e}")
        return None

### 🎯 Exercise 1: Search MusicBrainz for Metadata

In [ ]:
# Example: Search for a recording
if MB_AVAILABLE:
    results = search_musicbrainz_recording(
        artist="The Beatles",
        title="Hey Jude"
    )
    
    if results:
        print(f"Found {len(results)} results:\n")
        df = pd.DataFrame(results)
        display(df[['title', 'artist', 'album', 'date', 'score']])
    else:
        print("No results found.")
else:
    print("MusicBrainz not available. Install musicbrainzngs to use this feature.")

In [ ]:
# Try with your own artist/song
# Modify these values:
my_artist = "Artist Name"
my_title = "Song Title"

# Uncomment to search:
# if MB_AVAILABLE:
#     results = search_musicbrainz_recording(my_artist, my_title)
#     if results:
#         df = pd.DataFrame(results)
#         display(df)

---

## Part 2: Album Art Download

Download album artwork from Cover Art Archive (part of MusicBrainz).

### Cover Art Archive

- Free, high-quality album artwork
- Multiple sizes available (250px, 500px, 1200px, original)
- Officially licensed and legal to use

In [ ]:
def get_cover_art_url(release_id: str, size: str = '500') -> Optional[str]:
    """
    Get Cover Art Archive URL for a release.
    
    Args:
        release_id: MusicBrainz release ID
        size: '250', '500', '1200', or 'original'
    
    Returns:
        URL to album artwork, or None if not available
    """
    if not REQUESTS_AVAILABLE:
        return None
    
    # Cover Art Archive URL format
    base_url = f"https://coverartarchive.org/release/{release_id}"
    
    try:
        # Check if cover art exists
        response = requests.get(base_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            
            # Get front cover
            for image in data.get('images', []):
                if image.get('front', False):
                    # Get appropriate size
                    thumbnails = image.get('thumbnails', {})
                    
                    if size in thumbnails:
                        return thumbnails[size]
                    else:
                        return image.get('image')  # Original
            
            # If no front cover marked, use first image
            if data.get('images'):
                return data['images'][0].get('image')
        
        return None
    
    except Exception as e:
        return None

In [ ]:
def download_cover_art(url: str, output_path: Path) -> bool:
    """
    Download album art from a URL and save to file.
    
    Returns True if successful.
    """
    if not REQUESTS_AVAILABLE:
        print("Requests library not available.")
        return False
    
    try:
        response = requests.get(url, timeout=30)
        
        if response.status_code == 200:
            # Create parent directory if needed
            output_path.parent.mkdir(parents=True, exist_ok=True)
            
            # Save image
            with open(output_path, 'wb') as f:
                f.write(response.content)
            
            return True
        else:
            print(f"Download failed: HTTP {response.status_code}")
            return False
    
    except Exception as e:
        print(f"Error downloading: {e}")
        return False

### 🎯 Exercise 2: Download Album Art

In [ ]:
# Example: Search for an album and download its cover art
if MB_AVAILABLE and REQUESTS_AVAILABLE:
    # Search for a recording
    results = search_musicbrainz_recording("The Beatles", "Hey Jude")
    
    if results and 'release_id' in results[0]:
        release_id = results[0]['release_id']
        print(f"Release ID: {release_id}")
        print(f"Album: {results[0].get('album', 'Unknown')}\n")
        
        # Get cover art URL
        cover_url = get_cover_art_url(release_id, size='500')
        
        if cover_url:
            print(f"Cover art URL: {cover_url}\n")
            
            # Download and display
            temp_path = Path('/tmp/test_cover.jpg')
            if download_cover_art(cover_url, temp_path):
                print("✓ Downloaded successfully!\n")
                display(IPythonImage(filename=str(temp_path)))
            else:
                print("✗ Download failed")
        else:
            print("No cover art available for this release.")
    else:
        print("No release ID found in search results.")
else:
    print("Required libraries not available.")

---

## Part 3: Automated Metadata Enrichment

Automatically enhance metadata for files in your library.

### The Workflow

1. **Read existing metadata** from your file
2. **Search MusicBrainz** using artist/title
3. **Compare results** and pick best match
4. **Enrich metadata** with official information
5. **Download album art** if missing
6. **Update the file** with new metadata

In [ ]:
def enrich_file_metadata(file_path: Path, dry_run: bool = True) -> Dict:
    """
    Enrich metadata for a single file using MusicBrainz.
    
    Args:
        file_path: Path to audio file
        dry_run: If True, only preview changes without modifying file
    
    Returns:
        Dictionary with enrichment results
    """
    result = {
        'file': str(file_path.name),
        'status': 'skipped',
        'changes': [],
        'errors': []
    }
    
    if not MB_AVAILABLE:
        result['errors'].append("MusicBrainz not available")
        return result
    
    try:
        # Read existing metadata
        audio = File(file_path, easy=True)
        
        if not audio:
            result['errors'].append("Cannot read file")
            return result
        
        # Extract current metadata
        current_artist = str(audio.get('artist', [''])[0]) if 'artist' in audio else ''
        current_title = str(audio.get('title', [''])[0]) if 'title' in audio else ''
        current_album = str(audio.get('album', [''])[0]) if 'album' in audio else ''
        current_date = str(audio.get('date', [''])[0]) if 'date' in audio else ''
        
        # Need at least artist or title to search
        if not current_artist and not current_title:
            result['errors'].append("Insufficient metadata to search")
            return result
        
        # Search MusicBrainz
        mb_results = search_musicbrainz_recording(current_artist, current_title, limit=3)
        
        if not mb_results:
            result['status'] = 'not_found'
            return result
        
        # Use best match (highest score)
        best_match = mb_results[0]
        result['mb_score'] = best_match.get('score', 0)
        
        # Only proceed if confidence is high enough (score > 80)
        if int(best_match.get('score', 0)) < 80:
            result['status'] = 'low_confidence'
            result['errors'].append(f"Match score too low: {best_match.get('score')}")
            return result
        
        # Compare and note changes
        changes = []
        
        mb_artist = best_match.get('artist', '')
        mb_title = best_match.get('title', '')
        mb_album = best_match.get('album', '')
        mb_date = best_match.get('date', '')
        
        if mb_artist and mb_artist != current_artist:
            changes.append(f"Artist: '{current_artist}' → '{mb_artist}'")
        
        if mb_title and mb_title != current_title:
            changes.append(f"Title: '{current_title}' → '{mb_title}'")
        
        if mb_album and mb_album != current_album:
            changes.append(f"Album: '{current_album}' → '{mb_album}'")
        
        if mb_date and not current_date:
            changes.append(f"Date: (empty) → '{mb_date}'")
        
        if changes:
            result['status'] = 'enriched'
            result['changes'] = changes
            
            # Apply changes if not dry run
            if not dry_run:
                if mb_artist:
                    audio['artist'] = mb_artist
                if mb_title:
                    audio['title'] = mb_title
                if mb_album:
                    audio['album'] = mb_album
                if mb_date and not current_date:
                    audio['date'] = mb_date
                
                audio.save()
        else:
            result['status'] = 'no_changes'
        
        return result
    
    except Exception as e:
        result['errors'].append(str(e))
        return result

In [ ]:
def batch_enrich_library(library_path: Path, max_files: int = 10, dry_run: bool = True) -> List[Dict]:
    """
    Batch enrich metadata for files in library.
    
    Args:
        library_path: Root library path
        max_files: Maximum number of files to process (rate limiting)
        dry_run: Preview changes without modifying files
    
    Returns:
        List of enrichment results
    """
    print(f"Enriching metadata from: {library_path}")
    print(f"Mode: {'DRY RUN (preview only)' if dry_run else 'LIVE (will modify files)'}")
    print(f"Max files: {max_files}\n")
    
    results = []
    processed = 0
    
    for root, dirs, files in os.walk(library_path):
        for file in files:
            if processed >= max_files:
                break
            
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                print(f"Processing: {file}...")
                result = enrich_file_metadata(file_path, dry_run=dry_run)
                results.append(result)
                processed += 1
                
                if result['changes']:
                    print(f"  Status: {result['status']}")
                    for change in result['changes']:
                        print(f"    - {change}")
                print()
        
        if processed >= max_files:
            break
    
    print(f"\n✓ Processed {processed} files")
    
    # Summary
    enriched = sum(1 for r in results if r['status'] == 'enriched')
    not_found = sum(1 for r in results if r['status'] == 'not_found')
    low_conf = sum(1 for r in results if r['status'] == 'low_confidence')
    
    print(f"Enriched: {enriched}")
    print(f"Not found: {not_found}")
    print(f"Low confidence: {low_conf}")
    
    return results

### 🎯 Exercise 3: Batch Enrich Metadata

**Note**: This uses API calls. Start with a small number of files!

In [ ]:
# Preview enrichment for up to 10 files
if MB_AVAILABLE:
    enrichment_results = batch_enrich_library(
        MUSIC_LIBRARY_PATH,
        max_files=10,
        dry_run=True  # Preview only
    )
    
    # Show results
    if enrichment_results:
        df = pd.DataFrame(enrichment_results)
        display(df[['file', 'status', 'changes']])
else:
    print("MusicBrainz not available.")

In [ ]:
# Uncomment to actually apply changes (WARNING: modifies files!)
# enrichment_results = batch_enrich_library(
#     MUSIC_LIBRARY_PATH,
#     max_files=10,
#     dry_run=False  # Actually modify files
# )

---

## Part 4: API Best Practices

How to use external APIs responsibly and effectively.

### Rate Limiting

**MusicBrainz allows 1 request per second** for anonymous users.

- Respect this limit to avoid getting blocked
- Use `time.sleep()` between requests
- Consider caching results

### Caching

We cache API results in memory to:
- Avoid redundant queries
- Speed up repeated operations
- Reduce API load

### Error Handling

APIs can fail for many reasons:
- Network issues
- Server downtime
- Rate limit exceeded
- Invalid requests

Always handle errors gracefully!

In [ ]:
def save_api_cache(cache_file: Path):
    """
    Save API cache to disk for reuse in future sessions.
    """
    try:
        cache_file.parent.mkdir(parents=True, exist_ok=True)
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(API_CACHE, f, indent=2)
        print(f"✓ Cache saved: {len(API_CACHE)} entries")
    except Exception as e:
        print(f"Error saving cache: {e}")


def load_api_cache(cache_file: Path):
    """
    Load API cache from disk.
    """
    global API_CACHE
    
    try:
        if cache_file.exists():
            with open(cache_file, 'r', encoding='utf-8') as f:
                API_CACHE = json.load(f)
            print(f"✓ Cache loaded: {len(API_CACHE)} entries")
        else:
            print("No cache file found. Starting fresh.")
    except Exception as e:
        print(f"Error loading cache: {e}")

### 🎯 Exercise 4: Cache Management

In [ ]:
# Save current cache to disk
cache_file = MUSIC_LIBRARY_PATH / '.api_cache.json'

if API_CACHE:
    save_api_cache(cache_file)
else:
    print("No cache to save.")

In [ ]:
# Load cache from disk
load_api_cache(cache_file)

print(f"\nCurrent cache has {len(API_CACHE)} entries")

---

## Summary

In this module, you learned:

### Key Concepts

1. **MusicBrainz Integration**
   - Searching for recordings and releases
   - Getting authoritative metadata
   - Understanding match confidence scores

2. **Album Art Download**
   - Accessing Cover Art Archive
   - Downloading different image sizes
   - Displaying and saving artwork

3. **Metadata Enrichment**
   - Automatically enhancing file metadata
   - Batch processing workflows
   - Dry-run for safe testing

4. **API Best Practices**
   - Respecting rate limits
   - Caching results
   - Error handling

### Best Practices

- **Always test with dry-run first** before modifying files
- **Start small**: Process 10-20 files initially to verify results
- **Check match scores**: Only use high-confidence matches (>80)
- **Respect rate limits**: 1 request/second for MusicBrainz
- **Save cache**: Persist API results to avoid redundant queries
- **Backup first**: Always backup before bulk metadata changes

### Important Notes

**Rate Limiting**:
- MusicBrainz: 1 request/second (we implement this automatically)
- Processing 100 files takes ~2-3 minutes minimum
- Be patient - this is to respect the free service

**Match Quality**:
- Score >90: Excellent match, safe to auto-apply
- Score 80-90: Good match, review recommended
- Score <80: Low confidence, manual review required

### What's Next?

- **[Module 06: Library Visualizations](06_library_visualizations.ipynb)** - Visualize your enhanced metadata
- **[Module 08: Quality Validation](08_quality_validation.ipynb)** - Ensure metadata integrity

### Additional Resources

- **MusicBrainz**: [https://musicbrainz.org/](https://musicbrainz.org/)
- **Cover Art Archive**: [https://coverartarchive.org/](https://coverartarchive.org/)
- **API Documentation**: [https://musicbrainz.org/doc/MusicBrainz_API](https://musicbrainz.org/doc/MusicBrainz_API)
- **Python Library**: [https://python-musicbrainzngs.readthedocs.io/](https://python-musicbrainzngs.readthedocs.io/)

---

**Fantastic work!** You can now leverage the power of online music databases to perfect your library metadata. 🎵